In [98]:
import gspread
import fairpy

account = gspread.service_account("credentials.json")
spreadsheet = account.open("FairDivision")
input = spreadsheet.worksheet("input")
print("Rows: ", input.row_count, "Cols: ", input.col_count)
rows = input.get_all_values()
print(rows)

Rows:  1000 Cols:  27
[['מפלגה  v  ', 'מנדטים v   משרד >', 'חוץ', 'בטחון', 'אוצר', 'בט"פ', 'משפטים', 'פנים', 'בריאות', '', 'סה"כ'], ['הליכוד', '32', '20', '20', '20', '10', '10', '10', '10', '', '100'], ['הציונות הדתית', '14', '10', '20', '10', '30', '20', '10', '20', '', '120'], ['ש"ס', '11', '5', '5', '20', '5', '10', '30', '20', '', '95'], ['אגודת ישראל', '7', '5', '5', '5', '5', '5', '10', '20', '', '55'], ['סה"כ', '64', '', '', '', '', '', '', '', '', '']]


In [99]:

try:
	output = spreadsheet.worksheet("output")
except gspread.WorksheetNotFound:
	output = spreadsheet.add_worksheet(title="output", rows=len(agents)+2, cols=len(items)+3)
	# TODO: change worksheet direction to RTL
	# I did not find here https://docs.gspread.org/en/latest/api/models/worksheet.html#id1   how to do this.
input_range = input.range(1, 1, len(rows), len(rows[0]))
output.update_cells(input_range)


{'spreadsheetId': '1tJPV-y-r1TAx5FqbrqecKPJMeKHTtIDeiYck8eLoGKY',
 'updatedRange': 'output!A1:K6',
 'updatedRows': 6,
 'updatedColumns': 11,
 'updatedCells': 66}

## Read input

In [106]:
items = rows[0][2:-1]  # remove agent names, entitlements, and total
items.remove('')
print("items: ", items)

items:  ['חוץ', 'בטחון', 'אוצר', 'בט"פ', 'משפטים', 'פנים', 'בריאות']


In [107]:
rows_of_agents = rows[1:-1]    # remove item names and total
agents = [row[0] for row in rows_of_agents]         
print("agents: ", agents)

agents:  ['הליכוד', 'הציונות הדתית', 'ש"ס', 'אגודת ישראל']


In [139]:
entitlements = [int(row[1]) for row in rows_of_agents]
total_entitlements = sum(entitlements)
map_agent_to_entitlement = {agents[i]: entitlements[i] for i in range(len(agents))}
print("entitlements: ", map_agent_to_entitlement, ", total: ", total_entitlements)

entitlements:  {'הליכוד': 32, 'הציונות הדתית': 14, 'ש"ס': 11, 'אגודת ישראל': 7} , total:  64


In [128]:
def row_to_prefs(row:list)->list:
	prefs_list = row[2:-1]   # Remove party name, party entitlement, and total
	prefs_dict = {}
	for o in range(len(items)):
		value = prefs_list[o]
		value = 0.0 if value=='' else float(value)
		prefs_dict[items[o]] = value
	return prefs_dict
raw_preferences = {row[0]: row_to_prefs(row) for row in rows_of_agents}
print("raw_preferences: ",raw_preferences)

raw_preferences:  {'הליכוד': {'חוץ': 20.0, 'בטחון': 20.0, 'אוצר': 20.0, 'בט"פ': 10.0, 'משפטים': 10.0, 'פנים': 10.0, 'בריאות': 10.0}, 'הציונות הדתית': {'חוץ': 10.0, 'בטחון': 20.0, 'אוצר': 10.0, 'בט"פ': 30.0, 'משפטים': 20.0, 'פנים': 10.0, 'בריאות': 20.0}, 'ש"ס': {'חוץ': 5.0, 'בטחון': 5.0, 'אוצר': 20.0, 'בט"פ': 5.0, 'משפטים': 10.0, 'פנים': 30.0, 'בריאות': 20.0}, 'אגודת ישראל': {'חוץ': 5.0, 'בטחון': 5.0, 'אוצר': 5.0, 'בט"פ': 5.0, 'משפטים': 5.0, 'פנים': 10.0, 'בריאות': 20.0}}


In [147]:
def normalized_prefs(prefs, new_sum):
	current_sum = sum(prefs.values())
	ratio = new_sum/current_sum
	return {item: value*ratio for item,value in prefs.items()}
	
normalized_preferences = {
	agent: normalized_prefs(prefs, total_entitlements) for agent,prefs in raw_preferences.items()
}
for agent,prefs in normalized_preferences.items():
	print(agent, ":\t\t", prefs, "\t\t", sum(prefs.values()))

הליכוד :		 {'חוץ': 12.8, 'בטחון': 12.8, 'אוצר': 12.8, 'בט"פ': 6.4, 'משפטים': 6.4, 'פנים': 6.4, 'בריאות': 6.4} 		 64.0
הציונות הדתית :		 {'חוץ': 5.333333333333333, 'בטחון': 10.666666666666666, 'אוצר': 5.333333333333333, 'בט"פ': 16.0, 'משפטים': 10.666666666666666, 'פנים': 5.333333333333333, 'בריאות': 10.666666666666666} 		 63.99999999999999
ש"ס :		 {'חוץ': 3.3684210526315788, 'בטחון': 3.3684210526315788, 'אוצר': 13.473684210526315, 'בט"פ': 3.3684210526315788, 'משפטים': 6.7368421052631575, 'פנים': 20.210526315789473, 'בריאות': 13.473684210526315} 		 64.0
אגודת ישראל :		 {'חוץ': 5.818181818181818, 'בטחון': 5.818181818181818, 'אוצר': 5.818181818181818, 'בט"פ': 5.818181818181818, 'משפטים': 5.818181818181818, 'פנים': 11.636363636363637, 'בריאות': 23.272727272727273} 		 64.0


In [148]:
entitlement_normalized_preferences = {
	agent: normalized_prefs(prefs, total_entitlements / map_agent_to_entitlement[agent]) for agent,prefs in raw_preferences.items()
}
for agent,prefs in entitlement_normalized_preferences.items():
	print(agent, ":\t\t", prefs, "\t\t", sum(prefs.values()))

הליכוד :		 {'חוץ': 0.4, 'בטחון': 0.4, 'אוצר': 0.4, 'בט"פ': 0.2, 'משפטים': 0.2, 'פנים': 0.2, 'בריאות': 0.2} 		 2.0
הציונות הדתית :		 {'חוץ': 0.38095238095238093, 'בטחון': 0.7619047619047619, 'אוצר': 0.38095238095238093, 'בט"פ': 1.1428571428571428, 'משפטים': 0.7619047619047619, 'פנים': 0.38095238095238093, 'בריאות': 0.7619047619047619} 		 4.571428571428571
ש"ס :		 {'חוץ': 0.3062200956937799, 'בטחון': 0.3062200956937799, 'אוצר': 1.2248803827751196, 'בט"פ': 0.3062200956937799, 'משפטים': 0.6124401913875598, 'פנים': 1.8373205741626795, 'בריאות': 1.2248803827751196} 		 5.818181818181818
אגודת ישראל :		 {'חוץ': 0.8311688311688312, 'בטחון': 0.8311688311688312, 'אוצר': 0.8311688311688312, 'בט"פ': 0.8311688311688312, 'משפטים': 0.8311688311688312, 'פנים': 1.6623376623376624, 'בריאות': 3.324675324675325} 		 9.142857142857142


In [149]:
allocation = fairpy.items.leximin_optimal_allocation(entitlement_normalized_preferences)
allocation

הליכוד gets { 100.0% of חוץ, 100.0% of בטחון, 100.0% of אוצר, 100.0% of משפטים, 20.156% of פנים, 13.335% of בריאות} with value 1.47.
הציונות הדתית gets { 100.0% of בט"פ, 42.541% of בריאות} with value 1.47.
ש"ס gets { 79.844% of פנים} with value 1.47.
אגודת ישראל gets { 44.124% of בריאות} with value 1.47.

In [150]:
map_agent_to_fractions = {agents[i]: allocation.bundles[i].fractions for i in range(len(agents))}
map_agent_to_fractions

{'הליכוד': array([ 1.        ,  1.        ,  1.        , -0.        ,  1.        ,
         0.20156449,  0.13334546]),
 'הציונות הדתית': array([-0.        , -0.        , -0.        ,  1.        , -0.        ,
        -0.        ,  0.42541386]),
 'ש"ס': array([-0.        , -0.        , -0.        , -0.        , -0.        ,
         0.79843551, -0.        ]),
 'אגודת ישראל': array([-0.        , -0.        , -0.        , -0.        , -0.        ,
        -0.        ,  0.44124068])}

## Create output

In [152]:
for i in range(len(agents)):
	bundle_i = map_agent_to_fractions[agents[i]]
	print(agents[i], ": ", bundle_i)
	for o in range(len(items)):
		fraction_i_o = map_agent_to_fractions[agents[i]][o]
		# print(fraction_i_o)
		output.update_cell(i+2, o+3, fraction_i_o)

הליכוד :  [ 1.          1.          1.         -0.          1.          0.20156449
  0.13334546]
הציונות הדתית :  [-0.         -0.         -0.          1.         -0.         -0.
  0.42541386]
ש"ס :  [-0.         -0.         -0.         -0.         -0.          0.79843551
 -0.        ]
אגודת ישראל :  [-0.         -0.         -0.         -0.         -0.         -0.
  0.44124068]


In [164]:
# Insert formula for computing the utilities:
utility_column = len(items)+4
output.update_cell(1, utility_column, "תועלת באחוזים")
output.update_cell(1, utility_column+1, "תועלת ביחס למנדטים")


for i in range(len(agents)):
	# output.update_cell(i+2, utility_column, utility_profile[i])
	row_num = i+2
	first_cell = gspread.utils.rowcol_to_a1(row_num, 3)
	last_cell = gspread.utils.rowcol_to_a1(row_num, len(items)+2)
	range_a1 = f"{first_cell}:{last_cell}"
	output.update_cell(row_num, utility_column, f"=SUMPRODUCT(input!{range_a1},output!{range_a1})/sum(input!{range_a1})")
	utility_cell = gspread.utils.rowcol_to_a1(row_num, utility_column)
	output.format(utility_cell, {"numberFormat": {"type": "PERCENT", "pattern": "##.#%"}})
	entitlement_cell = gspread.utils.rowcol_to_a1(row_num, 2)
	output.update_cell(row_num, utility_column+1, f"={utility_cell}/{entitlement_cell}*100")
	# utility_per_entitlement_cell = gspread.utils.rowcol_to_a1(row_num, utility_column+1)
	# output.format(utility_per_entitlement_cell, {"numberFormat": {"type": "PERCENT", "pattern": "##.#%"}})